# Scrapping des infos utiles sur les animés

In [1]:
# Importation des packages nécessaires
import scrapy
import pandas as pd
import numpy as np
import pymongo
import selenium
import time
import os
import json
import datetime
import pymongo

client = pymongo.MongoClient("mongo")
database = client.anime
collection = database.myanimelist
#collection.drop()
client.database_names()

<ipython-input-1-fcfeca470ed2>:17: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  client.database_names()


['admin', 'config', 'local']

In [3]:
# Création du projet Scrappy nommé animescrawler
!scrapy startproject animescrawler

Error: scrapy.cfg already exists in /home/lcplacide/Documents/DataEngineerTools/6Evaluation/Projet/animescrawler


In [4]:
# Création de la spider myanimelist pour scrapper myanimelist.net
!cd animescrawler && scrapy genspider myanimelist myanimelist.net

Created spider 'myanimelist' using template 'basic' in module:
  {spiders_module.__name__}.{module}


## 1. Récupérer le menu de navigation horizontal de la page https://myanimelist.net/anime.php (#,A,B,....Z)

In [ ]:
scrapy shell 'https://myanimelist.net/anime.php'
response.css("#horiznav_nav").css("a::text").extract()
In [18]: response.css("#horiznav_nav").css("a::attr(href)").extract()
Out[18]: 
['https://myanimelist.net/anime.php?em=0&ed=0&ey=0&o=2&c%5B0%5D=a&c%5B1%5D=d&cv=1&w=0&sd=24&sm=11&sy=2020',
 'https://myanimelist.net/anime.php?o=9&c%5B0%5D=a&c%5B1%5D=d&cv=2&w=1',
 'https://myanimelist.net/anime.php?letter=.',
 'https://myanimelist.net/anime.php?letter=A',
 'https://myanimelist.net/anime.php?letter=B',
 'https://myanimelist.net/anime.php?letter=C',
 'https://myanimelist.net/anime.php?letter=D',
 'https://myanimelist.net/anime.php?letter=E',
 'https://myanimelist.net/anime.php?letter=F',
 'https://myanimelist.net/anime.php?letter=G',
 'https://myanimelist.net/anime.php?letter=H',
 'https://myanimelist.net/anime.php?letter=I',
 'https://myanimelist.net/anime.php?letter=J',
 'https://myanimelist.net/anime.php?letter=K',
 'https://myanimelist.net/anime.php?letter=L',
 'https://myanimelist.net/anime.php?letter=M',
 'https://myanimelist.net/anime.php?letter=N',
 'https://myanimelist.net/anime.php?letter=O',
 'https://myanimelist.net/anime.php?letter=P',
 'https://myanimelist.net/anime.php?letter=Q',
 'https://myanimelist.net/anime.php?letter=R',
 'https://myanimelist.net/anime.php?letter=S',
 'https://myanimelist.net/anime.php?letter=T',
 'https://myanimelist.net/anime.php?letter=U',
 'https://myanimelist.net/anime.php?letter=V',
 'https://myanimelist.net/anime.php?letter=W',
 'https://myanimelist.net/anime.php?letter=X',
 'https://myanimelist.net/anime.php?letter=Y',
 'https://myanimelist.net/anime.php?letter=Z']

## 2. Récupération des pages associées à chaque élément du menu de navigation

In [ ]:
scrapy shell 'https://myanimelist.net/anime.php?letter=A'
nb_page_A = int(response.css(".normal_header").css("a::text").extract()[-1])
last_link_page_A = response.urljoin(response.css(".normal_header").css("a::attr(href)").extract()[-1]) 
link_common_part_A = last_link_page_A.replace("show="+str((nb_page_A-1)*50),"show=")
all_link_page_A = [ link_common_part_A+str(i) for i in range(0,nb_page_A*50,50)]
Out[26]: 
['https://myanimelist.net/anime.php?letter=A&show=0',
 'https://myanimelist.net/anime.php?letter=A&show=50',
 'https://myanimelist.net/anime.php?letter=A&show=100',
 'https://myanimelist.net/anime.php?letter=A&show=150',
 'https://myanimelist.net/anime.php?letter=A&show=200',
 'https://myanimelist.net/anime.php?letter=A&show=250',
 'https://myanimelist.net/anime.php?letter=A&show=300',
 'https://myanimelist.net/anime.php?letter=A&show=350',
 'https://myanimelist.net/anime.php?letter=A&show=400',
 'https://myanimelist.net/anime.php?letter=A&show=450',
 'https://myanimelist.net/anime.php?letter=A&show=500',
 'https://myanimelist.net/anime.php?letter=A&show=550',
 'https://myanimelist.net/anime.php?letter=A&show=600',
 'https://myanimelist.net/anime.php?letter=A&show=650',
 'https://myanimelist.net/anime.php?letter=A&show=700',
 'https://myanimelist.net/anime.php?letter=A&show=750',
 'https://myanimelist.net/anime.php?letter=A&show=800',
 'https://myanimelist.net/anime.php?letter=A&show=850',
 'https://myanimelist.net/anime.php?letter=A&show=900']

## 3. Récupération des pages spécifiques à chaque animé

In [ ]:
scrapy shell 'https://myanimelist.net/anime.php?letter=A&show=0'
anime_link_A1 = set(response.css(".hoverinfo_trigger").css("a::attr(href)").extract())
anime_link_A1=response.xpath("//a[@class='hoverinfo_trigger fw-b fl-l']").css("::attr(href)").extract()

Out[18]: 
{'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials',
 'https://myanimelist.net/anime/10737/A-Channel___A-Channel',
 'https://myanimelist.net/anime/11339/A-Channel_smile',
 'https://myanimelist.net/anime/1295/A-Ko_The_Versus',
 'https://myanimelist.net/anime/1349/AD_Police_TV',
 'https://myanimelist.net/anime/17501/Abe_George_Kattobi_Seishun_Ki__Shibuya_Honky_Tonk',
 'https://myanimelist.net/anime/2198/Aa_Megami-sama__Sorezore_no_Tsubasa_Specials',
 'https://myanimelist.net/anime/22831/Abarenbou_Rikishi_Matsutarou',
 'https://myanimelist.net/anime/25397/Absolute_Duo',
 'https://myanimelist.net/anime/2884/Aachi_wa_Ssipak',
 'https://myanimelist.net/anime/29750/AAA_de_Ikou__Yuuna___Akiko',
 'https://myanimelist.net/anime/29902/A_Play',
 'https://myanimelist.net/anime/30214/A_Smart_Experiment',
 'https://myanimelist.net/anime/303/Aa_Megami-sama__Chichaitte_Koto_wa_Benri_da_ne',
 'https://myanimelist.net/anime/304/Aa_Megami-sama_Movie',
 'https://myanimelist.net/anime/305/Abashiri_Ikka',
 'https://myanimelist.net/anime/306/Abenobashi_Mahou☆Shoutengai',
 'https://myanimelist.net/anime/3090/Aa_Megami-sama__Tatakau_Tsubasa',
 'https://myanimelist.net/anime/320/A_Kite',
 'https://myanimelist.net/anime/32484/A-Size_Classmate',
 'https://myanimelist.net/anime/33337/ACCA__13-ku_Kansatsu-ka',
 'https://myanimelist.net/anime/3463/ALICE',
 'https://myanimelist.net/anime/35285/A-Channel__Nabe_wo_Tabeyou',
 'https://myanimelist.net/anime/36039/AICO__Incarnation',
 'https://myanimelist.net/anime/36182/Abstract_feat_Alkama',
 'https://myanimelist.net/anime/37180/A_Tang_Qi_Yu',
 'https://myanimelist.net/anime/37452/A_Woman_in_a_Fashion_Building',
 'https://myanimelist.net/anime/38701/Aa___ii',
 'https://myanimelist.net/anime/38712/A_Log_Day_of_Timbre',
 'https://myanimelist.net/anime/39057/A_New_Journey',
 'https://myanimelist.net/anime/39086/A_Christmas_Song',
 'https://myanimelist.net/anime/39184/A3_Season_Spring___Summer',
 'https://myanimelist.net/anime/39997/Ab-rah',
 'https://myanimelist.net/anime/40272/A3_Season_Autumn___Winter',
 'https://myanimelist.net/anime/40628/A_Brightening_Life',
 'https://myanimelist.net/anime/41319/A_song_for_You_You_You',
 'https://myanimelist.net/anime/41449/A_Sea_Doesnt_Tell_Much',
 'https://myanimelist.net/anime/42481/A_Day_Before_Us',
 'https://myanimelist.net/anime/44040/Abciee_Shuugyou_Nikki',
 'https://myanimelist.net/anime/49/Aa_Megami-sama',
 'https://myanimelist.net/anime/4934/AF',
 'https://myanimelist.net/anime/50/Aa_Megami-sama_TV',
 'https://myanimelist.net/anime/5447/A-Girl',
 'https://myanimelist.net/anime/5877/Abunai_Sisters__Koko___Mika',
 'https://myanimelist.net/anime/6094/A_Piece_of_Phantasmagoria',
 'https://myanimelist.net/anime/8216/Aa_Harimanada',
 'https://myanimelist.net/anime/880/Aa_Megami-sama__Sorezore_no_Tsubasa',
 'https://myanimelist.net/anime/9433/A-jangcom',
 'https://myanimelist.net/anime/9611/Aa_Megami-sama_2011',
 'https://myanimelist.net/anime/9776/A-Channel'}

## 4. Récupération des infos utiles de chaque animé
- Main title
- Alternative title
- Score
- Synopsis
- Ranked
- Popularity
- Genres
- Producers
- Licensors
- Studios
- Type
- Related animes
- Duration
- Rating
- Episode
- Status

In [ ]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
main_title = response.css("h1").css("strong::text").extract_first()
                          
Out[26]: 'Aa! Megami-sama! (TV) Specials'

In [ ]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
other_titles= response.xpath("//div[@style='width: 225px']/div[@class='spaceit_pad']/text()").extract()
other_titles = [elt.strip() for elt in alternative_titles if elt.strip()!='']

Out[32]: ['Ah! My Goddess! Specials, Aa! Megami-sama! Episodes 12.5, 25 and 26, Oh! My Goddess Specials', 'ああっ女神さまっ']

In [ ]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
score = response.css(".score-label::text").extract_first()

Out[34]: '7.54'

In [ ]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
synopsis = response.xpath("//p[@itemprop='description']").css("::text").extract()

Out[36]: 'Due to the recent events that happened on the last episodes of the season, the Goddess have some stability problems with their body systems. The always adult looking Urd becomes a little girl and has a fateful encounter with a boy, whereas little Skuld becomes an adult and wants to experiment adulthood together with Keiichi.'

In [ ]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
ranked = response.xpath("//div[@class='di-ib ml12 pl20 pt8']/span[@class='numbers ranked']/strong/text()").extract_first()
popularity = response.xpath("//div[@class='di-ib ml12 pl20 pt8']/span[@class='numbers popularity']/strong/text()").extract_first()

Out[39]: ('#1447', '#3253')

In [ ]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
genres = response.xpath("//span[@itemprop='genre']/text()").extract()

Out[42]: ['Magic', 'Comedy', 'Romance', 'Supernatural', 'Seinen']

In [ ]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
producers_licensors_studios = response.xpath("//div[@style='width: 225px']").xpath("//a[contains(@href,'/anime/producer')]/text()").extract()

Out[45]: ['Bandai Visual', 'AIC', 'Media Blasters', 'NYAV Post']

In [ ]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
table= response.xpath("//table[@class='anime_detail_related_anime']/tr")
related_anime= dict([(tr.css("td::text").extract_first(),tr.css("td a::text").extract()) for tr in table])

Out[126]: {'Adaptation': ['Aa! Megami-sama!'], 'Parent story': ['Aa! Megami-sama! (TV)']}

In [19]:
scrapy shell 'https://myanimelist.net/anime/1003/Aa_Megami-sama_TV_Specials'
border = response.xpath("//div[@style='width: 225px']")
divs= [div for div in border.css("div") if div.css("span::text").extract_first() in ["Duration:","Rating:","Type:","Episodes:","Status"]]
border_infos=dict([(div.css("span::text").extract_first(),div.css("::text").extract()[-1])for div in divs])

Out[4]: 
{'Type:': 'Special',
 'Duration:': '\n  24 min. per ep.\n  ',
 'Rating:': '\n  PG-13 - Teens 13 or older\n  '}

## Implémentation de la spider

In [ ]:
# JSON généré par scrapy
2020-12-14 21:27:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/28979/To_LOVE-Ru_Darkness_2nd>
{'Type': 'TV',
 'duration': datetime.time(0, 23),
 'episodes': 12,
 'genres': ['Comedy',
            'Ecchi',
            'Harem',
            'Romance',
            'School',
            'Sci-Fi',
            'Shounen'],
 'main_title': 'To LOVE-Ru Darkness 2nd',
 'other_titles': {'To LOVE Ru Darkness 2',
                  'To LOVE-Ru Trouble Darkness 2nd',
                  'To LOVEる -とらぶる- ダークネス2nd'},
 'popularity': 756,
 'producers': {'Magic Capsule',
               'NBCUniversal Entertainment Japan',
               'Sentai Filmworks',
               'Shueisha',
               'TBS',
               'Warner Bros. Japan',
               'Xebec'},
 'ranked': 1513,
 'rating': 'R+ - Mild Nudity',
 'related_anime': {'Adaptation': ['To LOVE-Ru Darkness'],
                   'Prequel': ['To LOVE-Ru Darkness'],
                   'Sequel': ['To LOVE-Ru Darkness 2nd Specials'],
                   'Side story': ['To LOVE-Ru Darkness OVA',
                                  'To LOVE-Ru Darkness 2nd OVA']},
 'score': 7.52,
 'status': 'Finished Airing',
 'synopsis': 'The dispassionate, transforming assassin Golden Darkness returns '
             'to peer deeper into the mysteries surrounding her new life, '
             'while a sinister Nemesis manipulates her younger sister Mea from '
             'the shadows. Along with their newly discovered mother, Tearju, '
             'this previously estranged family quickly becomes the center of '
             "everyone's attention. On the other hand, Princess Momo's Harem "
             "Plan stands on shaky ground amidst Rito's inability to confess "
             'to his longtime crush Haruna, who has grown feelings of her own. '
             "But things aren't as peaceful as they seem; an evil force looms "
             'amidst the innocuous commotion, threatening to eclipse the love, '
             'happiness, and friendship of Rito and his harem. Only the light '
             'of love can hope to banish the shadow. [Written by MAL Rewrite]'}

In [ ]:
# Résultat final attendu (version parsant aux max 20 pages par lettres)
{'downloader/request_bytes': 5797405,
 'downloader/request_count': 15546, # Nombre total de pages à parcourir: 15546
 'downloader/request_method_count/GET': 15546,
 'downloader/response_bytes': 1496636872,
 'downloader/response_count': 15546,
 'downloader/response_status_count/200': 15544, 
 'downloader/response_status_count/301': 2,
 'dupefilter/filtered': 1069,
 'elapsed_time_seconds': 2724.392909, # Temps d'exécution: 45 min environ
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 12, 14, 20, 27, 45, 436352),
 'item_scraped_count': 15178, # Nombre total d'animés: 15178
 'log_count/DEBUG': 30725,
 'log_count/INFO': 55,
 'memusage/max': 129540096,
 'memusage/startup': 77524992,
 'request_depth_max': 3,
 'response_received_count': 15544,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/200': 1,
 'scheduler/dequeued': 15544,
 'scheduler/dequeued/memory': 15544,
 'scheduler/enqueued': 15544,
 'scheduler/enqueued/memory': 15544,
 'start_time': datetime.datetime(2020, 12, 14, 19, 42, 21, 43443)}
2020-12-14 21:27:45 [scrapy.core.engine] INFO: Spider closed (finished)

In [ ]:
# Résultat final attendu (version parsant toutes les pages de chaque lettre)
{'downloader/request_bytes': 7278941,
 'downloader/request_count': 18123,
 'downloader/request_method_count/GET': 18123,
 'downloader/response_bytes': 1788362279,
 'downloader/response_count': 18123,
 'downloader/response_status_count/200': 18115,
 'downloader/response_status_count/301': 2,
 'downloader/response_status_count/403': 6,
 'dupefilter/filtered': 16537,
 'elapsed_time_seconds': 1753.178215,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 12, 21, 15, 15, 1, 329264),
 'item_scraped_count': 17370,
 'log_count/DEBUG': 35556,
 'log_count/INFO': 40,
 'memusage/max': 163659776,
 'memusage/startup': 66338816,
 'request_depth_max': 20,
 'response_received_count': 18121,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/200': 1,
 'scheduler/dequeued': 18121,
 'scheduler/dequeued/memory': 18121,
 'scheduler/enqueued': 18121,
 'scheduler/enqueued/memory': 18121,
 'start_time': datetime.datetime(2020, 12, 21, 14, 45, 48, 151049)}
2020-12-21 11:15:01 [scrapy.core.engine] INFO: Spider closed (finished)

In [13]:
!cd animescrawler/animescrawler/spiders && scrapy runspider myanimelist.py -o anime.json

2020-12-22 21:08:20 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: animescrawler)
2020-12-22 21:08:20 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.6 (default, Oct 13 2020, 20:37:26) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.2.1, Platform Linux-5.4.0-58-generic-x86_64-with-glibc2.2.5
2020-12-22 21:08:20 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-22 21:08:20 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'animescrawler',
 'COOKIES_ENABLED': False,
 'DOWNLOAD_DELAY': 0.01,
 'NEWSPIDER_MODULE': 'animescrawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_LOADER_WARN_ONLY': True,
 'SPIDER_MODULES': ['animescrawler.spiders']}
2020-12-22 21:08:20 [scrapy.extensions.telnet] INFO: Telnet Password: 3970283b000221bf
2020-12-22 21:08:20 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats

2020-12-22 21:08:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myanimelist.net/anime.php?letter=I&show=350> (referer: https://myanimelist.net/anime.php?letter=I)
2020-12-22 21:08:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/37352/Quote>
{'Type': 'Music',
 'duration': datetime.datetime(2020, 12, 22, 0, 4),
 'episodes': 3,
 'genres': ['Music'],
 'main_title': 'Quote',
 'other_titles': ['Quote', 'Red, Cambell, Amador'],
 'popularity': 11714,
 'producers': [],
 'ranked': 7919,
 'rating': 'G - All Ages',
 'related_anime': {'Other': ['Road to You']},
 'score': 6.06,
 'status': 'Finished Airing',
 'synopsis': 'Three official music videos for Red , Cambell , and Amador on '
             'Suda Keina\'s "Quote" album that was released on January 31, '
             '2018. Avogado6 animated them and they were released on Suda '
             "Keina's official YouTube channel as well as airing on Space "
             "Shower TV under U&R Records' label.

2020-12-22 21:08:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/29835/47_Todoufuken_Specials>
{'Type': 'Special',
 'duration': datetime.datetime(2020, 12, 22, 0, 4),
 'episodes': 2,
 'genres': ['Kids', 'Comedy', 'Romance'],
 'main_title': '47 Todoufuken Specials',
 'other_titles': ['47 Todoufuken 2012 Natsu, 47 Todoufuken 2012 Summer, 47 '
                  'Todoufuken Spring Special, 47 Todoufuken Summer Special',
                  '47都道府犬 2012夏'],
 'popularity': 13759,
 'producers': ['Pollyanna Graphics'],
 'ranked': 12987,
 'rating': 'G - All Ages',
 'related_anime': {'Parent story': ['47 Todoufuken']},
 'score': None,
 'status': 'Finished Airing',
 'synopsis': 'A two-part TV special focusing on the romance between the dogs.'}
2020-12-22 21:08:29 [scrapy.dupefilters] DEBUG: Filtered duplicate request: <GET https://myanimelist.net/anime.php?letter=M&show=950> - no more duplicates will be shown (see DUPEFILTER_DEBUG to show all duplicates)
2020-12-22 2

2020-12-22 21:08:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myanimelist.net/anime/38473/Itsuka_Aeru_Kimi_ni> (referer: https://myanimelist.net/anime.php?letter=I&show=400)
2020-12-22 21:08:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myanimelist.net/anime/41514/Itai_no_wa_Iya_nano_de_Bougyoryoku_ni_Kyokufuri_Shitai_to_Omoimasu_II> (referer: https://myanimelist.net/anime.php?letter=I&show=400)
2020-12-22 21:08:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/34542/Inuyashiki>
{'Type': 'TV',
 'duration': datetime.datetime(2020, 12, 22, 0, 22),
 'episodes': 11,
 'genres': ['Action', 'Drama', 'Psychological', 'Sci-Fi', 'Seinen'],
 'main_title': 'Inuyashiki',
 'other_titles': ['いぬやしき', 'Inuyashiki: Last Hero'],
 'popularity': 315,
 'producers': ['Aniplex',
               'Fuji TV',
               'DMM pictures',
               'MAPPA',
               'Dentsu',
               'Kodansha'],
 'ranked': 974,
 'rating': 'R - 17+ (violence

2020-12-22 21:08:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myanimelist.net/anime/19283/Oyako_Rankan_The_Animation> (referer: https://myanimelist.net/anime.php?letter=O&show=600)
2020-12-22 21:08:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/19283/Oyako_Rankan_The_Animation>
{'Type': 'OVA',
 'duration': datetime.datetime(2020, 12, 22, 0, 28),
 'episodes': 1,
 'genres': ['Hentai'],
 'main_title': 'Oyako Rankan The Animation',
 'other_titles': ['母娘乱館 THE ANIMATION', 'Ran Kan The Animation'],
 'popularity': 7616,
 'producers': ['Pink Pineapple'],
 'ranked': None,
 'rating': 'Rx - Hentai',
 'related_anime': {},
 'score': 6.23,
 'status': 'Finished Airing',
 'synopsis': 'Deep within a secluded village lies the Kamishiro household, a '
             'prestigious family whose women possess unmatched beauty. After '
             'being brought into the household following the death of his '
             'parents, Mamoru vows to repay the kindness

...

2020-12-22 21:08:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/22893/Mon_Cheri_CoCo>
{'Type': 'TV',
 'duration': datetime.datetime(2020, 12, 22, 0, 25),
 'episodes': 13,
 'genres': ['Romance', 'Shoujo'],
 'main_title': 'Mon Cheri CoCo',
 'other_titles': ['Mon Chéri Coco', 'モンシェリCOCO'],
 'popularity': 11922,
 'producers': ['TBS'],
 'ranked': 15479,
 'rating': 'G - All Ages',
 'related_anime': {'Adaptation': ['Mon Cheri CoCo']},
 'score': None,
 'status': 'Finished Airing',
 'synopsis': 'A young woman aims to become a famous fashion designer. (Source: '
             'AniDB)'}
2020-12-22 21:08:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myanimelist.net/anime/35465/Mon-Soni__Senritsu_no_Lucifer_Tada_Hitotsu_no_Hajimari_no_Uta> (referer: https://myanimelist.net/anime.php?letter=M&show=1150)
2020-12-22 21:08:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myanimelist.net/anime/9797/MonHun_Nikki_Girigiri_Airou_Mura__Airou_Kiki_Ippatsu_

2020-12-22 21:08:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/35153/Momotarou_no_Koutsuu_Anzen_Kyoushitsu>
{'Type': 'OVA',
 'duration': datetime.datetime(2020, 12, 22, 0, 12),
 'episodes': 1,
 'genres': ['Kids', 'Fantasy'],
 'main_title': 'Momotarou no Koutsuu Anzen Kyoushitsu',
 'other_titles': ['桃太郎の交通安全教室'],
 'popularity': 16943,
 'producers': [],
 'ranked': 15477,
 'rating': 'G - All Ages',
 'related_anime': {},
 'score': None,
 'status': 'Finished Airing',
 'synopsis': 'A Momotarou themed traffic safety video.'}
2020-12-22 21:08:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myanimelist.net/anime/10802/MonHun_Nikki_Girigiri_Airou_Mura_G> (referer: https://myanimelist.net/anime.php?letter=M&show=1150)
2020-12-22 21:08:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myanimelist.net/anime/37095/Violet_Evergarden__Kitto_Ai_wo_Shiru_Hi_ga_Kuru_no_Darou> (referer: https://myanimelist.net/anime.php?letter=V&show=50)
2020-12-22 21:08

2020-12-22 21:08:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/201/Video_Girl_Ai>
{'Type': 'OVA',
 'duration': datetime.datetime(2020, 12, 22, 0, 29),
 'episodes': 6,
 'genres': ['Comedy', 'Drama', 'Ecchi', 'Romance'],
 'main_title': 'Video Girl Ai',
 'other_titles': ["Den'ei Shoujo Video Girl AI",
                  'Video Girl Ai',
                  '電影少女 VIDEO・GIRL・AI'],
 'popularity': 3368,
 'producers': ['Shueisha', 'Production I.G'],
 'ranked': 2715,
 'rating': 'R+ - Mild Nudity',
 'related_anime': {'Adaptation': ['Video Girl Ai']},
 'score': 7.21,
 'status': 'Finished Airing',
 'synopsis': 'After a crushing rejection, the heartbroken Youta Moteuchi '
             'trudges home. On his way, a dimly lit video store catches his '
             'eye, and he ends up purchasing a "video girl" tape meant to ease '
             'the hearts of lonely men. However, when he tries to play it in '
             'his broken VCR, a beautiful girl leaps out of his

2020-12-22 21:08:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/823/Virgin_Night>
{'Type': 'OVA',
 'duration': datetime.datetime(2020, 12, 22, 0, 30),
 'episodes': 1,
 'genres': ['Hentai', 'Romance'],
 'main_title': 'Virgin Night',
 'other_titles': ['Shoya -Virgin Night-, First Night', '初夜 -ヴァージン・ナイト-'],
 'popularity': 7291,
 'producers': ['Green Bunny'],
 'ranked': None,
 'rating': 'Rx - Hentai',
 'related_anime': {'Adaptation': ['Virgin Night']},
 'score': 6.43,
 'status': 'Finished Airing',
 'synopsis': 'Azumi and Yuzuru are two young people who have been in a serious '
             'relationship for a while. Yuzuru wants to become more intimate '
             "with Azumi, but she isn't quite ready, so they go to a video "
             'store, unsure of what to do next. Finally, on the way back to '
             "Azumi's place, Azumi tells Yuzuru that tonight, she doesn't have "
             'to go home ... (Source: AniDB)'}
2020-12-22 21:08:48 [scra

2020-12-22 21:08:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://myanimelist.net/anime/30909/Vivi>
{'Type': 'Music',
 'duration': datetime.datetime(2020, 12, 22, 0, 4),
 'episodes': 1,
 'genres': ['Adventure', 'Music', 'Romance', 'Fantasy'],
 'main_title': 'Vivi',
 'other_titles': ['vivi'],
 'popularity': 11463,
 'producers': [],
 'ranked': 8254,
 'rating': 'G - All Ages',
 'related_anime': {'Other': ['Koi to Byounetsu']},
 'score': 5.96,
 'status': 'Finished Airing',
 'synopsis': 'A song from a concept album about an imaginary town that is '
             'built on top of a catfish. All of the songs are stories that '
             'happen in the town. The idea of having a miniature town was one '
             "of Yonezu's first when planning the album, and he created "
             'characters and stories for the people who lived in the town. '
             'Yonezu considers the lyrics of "vivi" to be the closest to his '
             'own feelings. (Source: Wikipedia)'}
2020

## Chargement des donneés dans mongo

In [2]:
with open("animescrawler/spiders/anime_complet2.json") as f:
    DOCUMENTS = json.load(f)
for i in range(len(DOCUMENTS)):
    DOCUMENTS[i]["_id"]=i
    if DOCUMENTS[i]["duration"]!=None:
        DOCUMENTS[i]["duration"]=datetime.datetime.strptime(DOCUMENTS[i]["duration"], '%Y-%m-%d %H:%M:%S')

collection.drop()
collection.insert_many(DOCUMENTS)
collection.find_one()

{'_id': 0,
 'main_title': 'Da Wei Bei Ken Zhi Daomei Tegong Xiong',
 'other_titles': ['Backkom Bear: Agent 008', '大卫贝肯之倒霉特工熊', 'Bernard Bear'],
 'score': None,
 'synopsis': 'A Chinese adaptation of the international Backkom franchise. This film serves as a prequel to the main show. With the human exploitation of natural resources, the Arctic environment is completely destroyed, leaving only the last polar bear to survive. A human girl named Jessica adopts the polar bear and names him Backkom. Jessica is an extraordinary agent of the Security Department. Backkom eager to become an agent someday, but he is so clumsy that he can only do the cleaning in the Security Department. All of a sudden, a mysterious organization of polar bears shows up, places bombs around the world, and claims to seek revenge from human race. To remove threat of bombs, the top decision-making body of men, Global League, decides to arrange agents going under the basement of the Polar Bear Organization. While there 

## Exploration des données

In [31]:
df=pd.read_json("animescrawler/spiders/anime_complet2.json",dtype={"duration":'datetime64[ns]'})
print(df.info(),df.describe(),sep=2*'\n')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17370 entries, 0 to 17369
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   main_title     17370 non-null  object        
 1   other_titles   17355 non-null  object        
 2   score          12178 non-null  float64       
 3   synopsis       16498 non-null  object        
 4   ranked         15621 non-null  float64       
 5   popularity     17370 non-null  int64         
 6   genres         17370 non-null  object        
 7   producers      17370 non-null  object        
 8   Type           17331 non-null  object        
 9   related_anime  17370 non-null  object        
 10  episodes       16850 non-null  float64       
 11  status         17370 non-null  object        
 12  duration       16820 non-null  datetime64[ns]
 13  rating         16697 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(9)
memory usage: 1.

,main_title,other_titles,score,synopsis,ranked,popularity,genres,producers,Type,related_anime,episodes,status,duration,rating
0,Da Wei Bei Ken Zhi Daomei Tegong Xiong,"[Backkom Bear: Agent 008, 大卫贝肯之倒霉特工熊, Bernard ...",NaN,A Chinese adaptation of the international Back...,13542.0,14725,"[Action, Police, Kids, Drama]",[],Movie,{},1.0,Finished Airing,2020-12-21 01:30:00,G - All Ages
1,Ero Manga! H mo Manga mo Step-up♪,[えろまんが! Hもマンガもステップアップ♪],6.80,Based on the erotic game by Clock Up.,NaN,5135,[Hentai],[Collaboration Works],OVA,{'Adaptation': ['Ero Manga! H mo Manga mo Step...,2.0,Finished Airing,2020-12-21 00:30:00,Rx - Hentai
2,Uchuu no Hou: Reimei-hen,"[The Laws of the Universe: Part 1, Happy Scien...",5.48,"University students, Ray, Anna, Tyler, Halle, ...",9599.0,9724,"[Sci-Fi, Space]",[HS Pictures Studio],Movie,{'Prequel': ['UFO Gakuen no Himitsu']},1.0,Finished Airing,2020-12-21 01:59:00,G - All Ages
3,Uchuu Ace,"[Space Ace, 宇宙エース, Uchu Ace, Uchuu Eesu, Uchu ...",NaN,Dr. Tatsunoko of Tatsunoko Research Center goe...,12455.0,12065,"[Action, Adventure, Sci-Fi]",[Tatsunoko Production],TV,{'Alternative version': ['Uchuu Ace Pilots']},52.0,Finished Airing,2020-12-21 00:25:00,G - All Ages
4,Hataraku Saibou!!,"[Cells at Work! 2nd Season, Hataraku Saibou 2n...",NaN,Second season of Hataraku Saibou .,NaN,1624,"[Comedy, Shounen]","[Aniplex, David Production]",TV,"{'Adaptation': ['Hataraku Saibou'], 'Prequel':...",NaN,Not yet aired,NaT,PG-13 - Teens 13 or older


In [4]:
# Localistion des données manquantes
df.isna().sum()

main_title          0
other_titles       15
score            5192
synopsis          872
ranked           1749
popularity          0
genres              0
producers           0
Type               39
related_anime       0
episodes          520
status              0
duration          550
rating            673
dtype: int64

In [22]:
# Valeurs prises par producers, genres, Type, status et rating
cols=["Type","status","rating"]
cols2=["genres","producers"]

for col in cols:
    print(df[col].value_counts(),end=2*"\n")
    
lg,lp=[],[]
for i in range(len(df)):
        lg= lg+df["genres"].iloc[i]
        lp= lp+df["producers"].iloc[i]
print(len(set(lg)),set(lg),end=2*"\n")
print(len(set(lp)),set(lp),end=2*"\n")

TV         4920
OVA        3883
Movie      3022
Special    2209
ONA        1841
Music      1456
Name: Type, dtype: int64

Finished Airing     16662
Not yet aired         381
Currently Airing      327
Name: status, dtype: int64

PG-13 - Teens 13 or older         6006
G - All Ages                      5764
PG - Children                     1449
Rx - Hentai                       1344
R - 17+ (violence & profanity)    1141
R+ - Mild Nudity                   993
Name: rating, dtype: int64

43 {'Military', 'Parody', 'Josei', 'Seinen', 'Harem', 'Horror', 'Space', 'Yuri', 'Super Power', 'Hentai', 'Magic', 'Shounen', 'Thriller', 'School', 'Comedy', 'Dementia', 'Shoujo Ai', 'Sci-Fi', 'Game', 'Kids', 'Supernatural', 'Vampire', 'Shoujo', 'Martial Arts', 'Psychological', 'Samurai', 'Adventure', 'Ecchi', 'Drama', 'Mystery', 'Action', 'Sports', 'Shounen Ai', 'Cars', 'Fantasy', 'Romance', 'Slice of Life', 'Demons', 'Mecha', 'Music', 'Police', 'Yaoi', 'Historical'}

1647 {'voque ting', 'Panasonic Digit

In [33]:
# Histogramme d'episodes, score, popularity, ranked, duration
import plotly.express as px
cols=["score","ranked","popularity","episodes","duration"]
for col in cols:
    px.histogram(df,col).show()